In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from skimage import io
import matplotlib.image
import os
from PIL import Image
from tqdm import tqdm

In [ ]:
def compress(origFilePath, compressedFilePath, dVals):
    im = io.imread(origFilePath, as_gray=True)
    # im -= np.average(im)

    u, d, vt = np.linalg.svd(im, full_matrices=False)

    smallu = u[0:, 0:dVals]
    smalld = np.diag(d[:dVals])
    smallvt = vt[0:dVals, 0:]
    compressedIm = smallu @ smalld @ smallvt

    fig, axes = plt.subplots(1, 2, figsize=(15, 10))
    ax = axes.ravel()

    ax[0].imshow(im, cmap=plt.cm.gray)
    #ax[1].plot(np.ma.log10(d), color='k')
    ax[1].imshow(compressedIm, cmap=plt.cm.gray)
    # ax[3].imshow(im-compressedIm, cmap=plt.cm.gray)

    print('compression Ratio: ', dVals/len(d))
    print('peak signal-to-noise: ', np.mean((im-compressedIm)**2))

    matplotlib.image.imsave(compressedFilePath, compressedIm, cmap=plt.cm.gray)

In [ ]:
def compressWithColor(origFilePath, compressedFilePath, dVals):
    #Read the image
    im = io.imread(origFilePath)
    #Create an array of same shape
    compressedIm = np.full(im.shape, 255)

    #Take the SVD of the red, green, and blue channels
    for i in range(3):
        col = im[ : , : , i]
        u, d, vt = np.linalg.svd(col, full_matrices=False)

        #Reconstruct each channel with only k singular values
        smallu = u[0:, 0:dVals]
        smalld = np.diag(d[:dVals])
        smallvt = vt[0:dVals, 0:]
        #Place the compressed channels in the compressed matrix
        compressedIm[ : , : , i] = smallu @ smalld @ smallvt


    #Make sure values are from 0-255 for all channels
    compressedIm = np.clip(compressedIm, 0, 255)
    #Change dtype to uint
    compressedIm = compressedIm.astype(np.uint8)

    #Display results
    fig, axes = plt.subplots(1, 2, figsize=(15, 10))
    ax = axes.ravel()

    ax[0].imshow(im)
    #ax[1].plot(np.ma.log10(d), color='k')
    ax[1].imshow(compressedIm)
    # ax[3].imshow(im-compressedIm, cmap=plt.cm.gray)

    #save the image
    matplotlib.image.imsave(compressedFilePath, compressedIm)

    im = im.astype(float)
    im/=255
    compressedIm = compressedIm.astype(float)
    compressedIm/=255
    pstn = np.mean((im-compressedIm)**2)
    print('compression Ratio: ', dVals/len(d))
    print('peak signal-to-noise: ', pstn)

In [ ]:
def compressWithPSTN(origFilePath, compressedFilePath, desiredpstn):
    #Read file
    im = io.imread(origFilePath)
    
    #Initialize the different channels
    red = im[ : , : , 0]
    green = im[ : , : , 1]
    blue = im[ : , : , 2]

    #Take the SVD of red, green, and blue channels
    red_u, red_d, red_vt = np.linalg.svd(red, full_matrices=False)
    green_u, green_d, green_vt = np.linalg.svd(green, full_matrices=False)
    blue_u, blue_d, blue_vt = np.linalg.svd(blue, full_matrices=False)

    #Place these SVDs in a list
    svds = [[red_u, red_d, red_vt], [green_u, green_d, green_vt], [blue_u, blue_d, blue_vt]]

    #Begin iteration at the maximum amount of dvalues
    dVals = int(len(red_d))
    #For binary search, increment begins at half the number of dvals
    increment = int(np.ceil(dVals/2))
    pstn = 0
    #Create an array of same shape as the original image
    compressedIm = np.full(im.shape, 255)

    #If increment is 1, that means binary search has converged
    while increment != 1:
        #Here we create the compressed image by taking the economy svd of each color channel.
        for i in range(3):
            compressedIm[ : , : , i] = svds[i][0][0:, 0:dVals] @ np.diag(svds[i][1][0:dVals]) @ svds[i][2][0:dVals, 0:]
            compressedIm = np.clip(compressedIm, 0, 255)

        #In order to determine peak signal-to-noise, all values must be between 0-1. Thus we divide by 255, take the pstn, and then revert
        im = im.astype(float)
        im/=255
        compressedIm = compressedIm.astype(float)
        compressedIm/=255
        pstn = np.mean((im-compressedIm)**2)
        im*=255
        np.round(im)
        im = im.astype(int)
        compressedIm*=255
        np.round(compressedIm)
        compressedIm = compressedIm.astype(int)

        #If we are less accurate than we want to be, increase the number of dvals
        if pstn > desiredpstn:
            dVals += increment
        #If we are more accurate than we need to be, decrease the number of dvals
        else:
            dVals -= increment
        increment = int(np.ceil(increment/2))
        if dVals < 1:
            dVals = 1
            break

    #Create the final compressed image with the converged values of dvals
    for i in range(3):
        compressedIm[ : , : , i] = svds[i][0][0:, 0:dVals] @ np.diag(svds[i][1][0:dVals]) @ svds[i][2][0:dVals, 0:]
    #Ensure all values are between 0 and 255
    compressedIm = np.clip(compressedIm, 0, 255)
    im = np.clip(im, 0, 255)


    #Show images
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    ax = axes.ravel()

    ax[0].imshow(im)
    ax[1].plot(np.ma.log10(red_d), color='k')
    ax[2].imshow(compressedIm)
    ax[3].imshow(np.abs(im-compressedIm)*10, cmap=plt.cm.gray) 

    #Save image
    #Convert to np.uint8 for saving
    compressedIm = compressedIm.astype(np.uint8)
    matplotlib.image.imsave(compressedFilePath, compressedIm)

    #Take the final PSTN
    im = im.astype(float)
    im/=255
    compressedIm = compressedIm.astype(float)
    compressedIm/=255
    pstn = np.mean((im-compressedIm)**2)


    ########
    # Print info
    print('compression Ratio: ', dVals/len(red_d))
    print('peak signal-to-noise: ', pstn)

    #########

def compressWithCompressionRatio(origFilePath, compressedFilePath, desiredCompressionRatio):
    #Load the original file
    im = io.imread(origFilePath)
    #Create numpy array of same shape
    compressedIm = np.full(im.shape, 255)

    #Take the SVD of each color channel
    for i in range(3):
        col = im[ : , : , i]
        u, d, vt = np.linalg.svd(col, full_matrices=False)
        #We use dvals proportional to what our desired compression ratio is.
        dVals = np.uint8(len(d) * desiredCompressionRatio)

        smallu = u[0:, 0:dVals]
        smalld = np.diag(d[:dVals])
        smallvt = vt[0:dVals, 0:]
        compressedIm[ : , : , i] = smallu @ smalld @ smallvt


    #Put a ceiling and floor on all the values
    compressedIm = np.clip(compressedIm, 0, 255)
    #Store as an int for residual
    compressedIm = compressedIm.astype(int)
    im = im.astype(int)

    #Show results
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    ax = axes.ravel()

    ax[0].imshow(im)
    ax[1].plot(np.ma.log10(d), color='k')
    ax[2].imshow(compressedIm)
    ax[3].imshow(np.abs(im-compressedIm)*10, cmap=plt.cm.gray) 

    #Save image
    compressedIm = compressedIm.astype(np.uint8)
    matplotlib.image.imsave(compressedFilePath, compressedIm)

    im = im.astype(float)
    im/=255
    compressedIm = compressedIm.astype(float)
    compressedIm/=255

    # Print info
    print('compression Ratio: ', dVals/len(d))
    print('peak signal-to-noise: ', np.mean((im-compressedIm)**2))


In [ ]:
def compareTwoImages(file1, file2):
    im = io.imread(file1)
    im2 = io.imread(file2)

    fig, axes = plt.subplots(1, 3, figsize=(15, 10))
    ax = axes.ravel()

    ax[0].imshow(im)
    ax[1].imshow(im2)
    ax[2].imshow(im-im2)

    file_stats1 = os.stat(file1)
    file_stats2 = os.stat(file2)

    print('compression ratio: ', file_stats2.st_size/file_stats1.st_size)
    print('peak signal-to-noise: ', np.mean((im-im2)**2))

In [ ]:
for filename in tqdm(os.listdir('pugImages/')):
    if filename[0] != '.':
        compressWithCompressionRatio(f'pugImages/{filename}', f'compressedPugs/{filename}', .08)

In [ ]:
for filename in tqdm(os.listdir('pugImages/')):
    if filename[0] != '.':
        compressWithPSTN(f'pugImages/{filename}', f'compressedPugs2/{filename}', .0025)

In [ ]:
for filename in tqdm(os.listdir('pugImages/')):
    if filename[0] != '.':
        compareTwoImages(f'pugImages/{filename}', f'compressedPugs/{filename}')

In [ ]:
compressWithColor('static/dog1.jpg', 'compressions/compressed3.jpg', 50)

In [ ]:
compressWithPSTN('result.png', 'compressions/result.png', .00025)

In [ ]:
compressWithCompressionRatio('static/dog1.jpg', 'compressions/dog1.jpg', .05)